# БЭГГИНГ (BAGGING, BOOTSTRAP AGGREGATING)

<span style="font-family:Papyrus; font-size:1.5em;">АНСАМБЛЬ МОДЕЛЕЙ (БЭГГИНГ)</span>

Bagging — это параллельный способ построения ансамбля. Коротко о способе построения:

- Обучающая выборка сэмплируется  раз с помощью bootstrap (выбор с возвращением).
- На каждом сэмпле обучается отдельная базовая модель.
- Ответы моделей усредняются (возможно, с весами).

Необходимость бэггинга может возникнуть тогда, когда вы уже нашли хорошую модель, и никак больше не можете повысить ее качество. В этом случае можно перейти к более продвинутой истории: использовать не одну модель (пусть и очень хорошую!), а ансамбли моделей. Под термином ансамбли понимается ряд продвинутых техник машинного обучения, о которых мы поговорим далее.

> ## ПРИМЕР: ТЕОРЕМА КОНДОРСА
Проиллюстрируем идею ансамблей на известном примере — теореме Кондорса о жюри присяжных, которая датируется аж 1784 годом!
>
> Представим, что у нас есть несколько членов жюри, мнение каждого из них независимо от мнения других. Мы не знаем, какая вероятность принятия верного решения у каждого члена жюри. Однако мы понимаем, что если вероятность принять правильное решение у какого-то члена жюри больше, чем , то и общая вероятность принять верное решение возрастает. И наоборот, если вероятность принять правильное решение у какого-то члена жюри меньше, чем , то общая вероятность принять верное решение падает.

## БЭГГИНГ (BAGGING, BOOTSTRAP AGGREGATING)

Цель бэггинга заключается в том, чтобы создать ансамбль из нескольких моделей. Такая ансамблевая модель будет надежнее, чем составляющие ее части.

В основе бэггинга лежит статистический метод, который называется бутстрэпом (bootstrap). Идея бутстрэпа заключается в генерации выборок размера **B** (так называемых бутстрэп-выборок) из исходного датасета размера **N** путем случайного выбора элементов с повторениями в каждом из наблюдений **B**.

Рассмотрим идею бутстрэпа на элементарном примере.

Пусть у нас есть выборка из **12** элементов. Тогда мы можем из нее выбирать различные выборки из нового количества элементов (в данном случае из 5). При этом, если мы использовали какой-то объект, то мы можем использовать его снова. Таким образом, мы можем из одной выборки получить множество новых.

![bagging](./../img/bagging1.png)

При некотором приближении можно считать, что получающиеся выборки являются независимыми и репрезентативными.

Такие бутстрэп-выборки часто используются для оценки различных статистических показателей (например, разброса или доверительного интервала). Если вычислять статистические оценки на нескольких независимых выборках, то мы можем оценить их разброс. Поиск большого количества независимых выборок сложен в силу того, что для этого требуется слишком много данных. Поэтому мы используем бутстрэп, чтобы создать несколько выборок, которые являются независимыми и репрезентативными (но, опять же, стоит отметить, что такими их можно считать только при нескольких допущениях).

Перейдем к понятию бэггинга. При построении моделей всегда присутствует вероятность, что при обучении на других данных мы получили бы другие результаты. Для того, чтобы нивелировать такую вероятность, можно использовать бэггинг. 

Его идея состоит в том, что мы берем несколько независимых моделей и усредняем полученные по ним результаты. Таким образом, мы получаем модель, имеющую меньший разброс, так как при ее построении мы учли несколько моделей. Как уже было сказано, в реальности получить много независимых выборок слишком сложно в силу того, что найти столько данных обычно не представляется возможным. Поэтому мы используем бутстрэп-выборки. 

> Важно отметить, что при бэггинге размер каждой бутстрэп-выборки должен совпадать с размером исходной выборки.

## RANDOM SUBSPACES (RSS)

Для построения набора различных моделей используется также метод выбора случайных подвыборок признаков Random Subspaces. Метод обеспечивает устойчивость алгоритма к набору доступных признаков.

## ЗАДАЧА О ВИНАХ
Реализуем бэггинг для деревьев решений. Для тренировки будем использовать датасет о винах. 

<div style="text-align: center;">
    <img src="./../img/wine.png" style="width: 300px;"/>
</div>

Датасет можно скачать [здесь winequality-red.csv](./../data).

Для начала подготовим данные к классификации. Условно разделим вино на хорошее и нет. Хорошим вином будем называть вино, параметр quality которого не менее 6.

Теперь сравним несколько методов классификации: логистическую регрессию, решающее дерево и бэггинг.

Разбейте выборку на обучающую и тренировочную с параметрами `test_size=0.30, random_state=42`.

Обучите два классификатора: логистическую регрессию (с дефолтными параметрами) и решающее дерево (`random_state=42`, максимальная глубина равна `10`).

In [1]:
import pandas as pd
import numpy as np

from sklearn import model_selection, datasets, metrics, tree, ensemble, linear_model

In [2]:
%run ./../model_inspector.ipynb

In [3]:
import warnings; warnings.simplefilter('ignore')

In [4]:
wine_df = pd.read_csv('./../data/winequality-red.csv', sep=';')

In [5]:
wine_df.sample(5)

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
416,10.6,0.48,0.64,2.2,0.111,6.0,20.0,0.99700,3.26,0.66,11.7,6
1435,10.2,0.54,0.37,15.4,0.214,55.0,95.0,1.00369,3.18,0.77,9.0,6
1372,8.7,0.78,0.51,1.7,0.415,12.0,66.0,0.99623,3.00,1.17,9.2,5
959,8.0,0.59,0.05,2.0,0.089,12.0,32.0,0.99735,3.36,0.61,10.0,5
836,6.7,0.28,0.28,2.4,0.012,36.0,100.0,0.99064,3.26,0.39,11.7,7


In [6]:
RANDOM_STATE = 42

In [7]:
# train test data
X_train, X_test, y_train, y_test = model_selection.train_test_split(
    wine_df.drop(columns=['quality']), 
    wine_df['quality'], 
    test_size=0.3, 
    random_state=RANDOM_STATE
)

In [8]:
log_reg_clf = linear_model.LogisticRegression()
log_reg_clf.fit(X_train, y_train)

LogisticRegression()

In [9]:
dec_tree_clf = tree.DecisionTreeClassifier(random_state=RANDOM_STATE, max_depth=10)
dec_tree_clf.fit(X_train, y_train)

DecisionTreeClassifier(max_depth=10, random_state=42)

### Задача 1

Введите значение f1 score для классификатора, который показал наилучшее значение.

In [10]:
log_reg_predictions = log_reg_clf.predict(X_test)
np.round(metrics.f1_score(y_test, log_reg_predictions), 3)

ValueError: Target is multiclass but average='binary'. Please choose another average setting, one of [None, 'micro', 'macro', 'weighted'].

In [ ]:
dec_tree_clf_predictions = dec_tree_clf.predict(X_test)
np.round(metrics.f1_score(y_test, dec_tree_clf_predictions), 3)

### Задача 2

Обучите модель с использование бэггинга (функция BaggingClassifier с `random_state=42`, разделение выборки на обучающую и тренировочную с параметрами `test_size=0.30, random_state=42`) для алгоритма, показавшего лучшее качество, определите количество моделей `1500`. Вычислите новое значение `f1-score`.

In [ ]:
tree_bagging_clf = ensemble.BaggingClassifier(dec_tree_clf, random_state=RANDOM_STATE, n_estimators=1500)
tree_bagging_clf.fit(X_train, y_train)

In [ ]:
tree_bagging_clf_predictions = tree_bagging_clf.predict(X_test)
np.round(metrics.f1_score(y_test, tree_bagging_clf_predictions, average='weighted'), 3)